In [ ]:
import time

from simulationClass import FIMS_Simulation

In [ ]:
def scanParam(parameters, minValues, maxValues, numSteps):
    """
    Executes a parameter scan by running the simulation multiple times with
    varying combinations of specified parameters.

    Executes an N-dimensional linear scan with every combination of parameters.
    If a single simulation fails, its run number is set to -1 and the scan continues.

    Args:
        parameters (list): List of strings for parameters to be scanned.
        minValues (list): List of numbers; Minimum value of each parameter.
        maxValues (list): List of numbers; Maximum value of each parameter.
        numSteps (list): List of ints; Number of steps within each parameter's range.

    Returns:
        list: List of the run numbers of the executed simulations.
              Empty if the parameters are not recognized.
    """

    if not (len(parameters) == len(minValues) == len(maxValues) == len(numSteps)):
        raise ValueError('Inputs must have the same length.')

    simulation = FIMS_Simulation()
    allParams = simulation.defaultParam()

    for inParam in parameters:
        if inParam not in allParam:
            print(f"Parameter '{inParam}' not able to be varied.")
            return []
    
    #Find range for each parameter
    allRanges = []
    for i, _ in enumerate(parameters):
        paramRange = np.linspace(minValues[i], maxValues[i], numSteps[i])
        allRanges.append(paramRange)

    print(f'Beginning scan through {parameters}. {math.prod(numSteps)} runs.')

    runNos = []
    for scanParams in itertools.product(*allRanges):

        #Reset parameters to defaults
        simulation.resetParam(varbose=False)
        
        #Apply current parameter combination
        for i, inParam in enumerate(parameters):
            simulation.param[inParam] = scanParams[i]

        runNo = simulation.runSimulation()
        if runNo == -1:
            print(f'Simulation failed with settings: {simulation}')
        runNos.append(runNo)

    print(f'Done scan through {parameters}. Final run number: {runNos[-1]}')
    simulation.resetParam()
        
    return runNos

In [ ]:
def scanValues(parameterDefinitions):
    """
    Executes a parameter scan by running the simulation multiple times with
    varying combinations of specified parameters.

    Executes an N-dimensional linear scan with every combination of parameters.
    If a single simulation fails, its run number is set to -1 and the scan continues.

    Args:
        parameterDefinitions (list): List of the following form: 
                                     ['parameter', minValue, maxValue, stepValue]
                                     'parameter' - String of the parameter name.
                                      minValue - Number for the minimum parameter value.
                                      maxValue - Number for the maximum parameter value.
                                      stepValue - Integer number of steps through range.

    Returns:
        list: List of the run numbers of the executed simulations.
              Empty if the parameters are not recognized.
    """
    #Separate the elements of scanning parameters
    parameters = []
    minValues = []
    maxValues = []
    numSteps = []
    
    for inDefinition in parameterDefinitions:
        if not (len(inDefinition) == 4):
            raise ValueError("Input list elements must have the form: ['parameter', minValue, maxValue, stepValue]")
        
        inParam, inMin, inMax, inStep = inDefinition
            
        parameters.append(inParam)
        minValues.append(inMin)
        maxValues.append(inMax)
        numSteps.append(inStep)

    #run the scan
    runNos = scanParam(parameters, minValues, maxValues, numSteps)
        
    return runNos

In [ ]:
def readRootTrees(runNumber: int) -> dict:
    """
    Reads all trees from a ROOT file and returns them as a dictionary.
    The keys are the tree names and the values are pandas DataFrames.

    Args:
        runNumber (int): The run number of the ROOT file.

    Returns:
        dict: A dictionary where keys are tree names (str) and values are pandas DataFrames. 
              Empty if the file cannot be opened or contains no trees.
    """
    
    #TODO - This function itself is named similarly to one in analysis.ipynb
    #    They rely on the location of the data folder.
    #    Likely solution - move data up the heirarchy, to exist at similar relative place. (../../ i think)
    dataFilePath = 'Data/' 
    dataFile = f'sim.{runNumber}.root'
    fullFilePath = os.path.join(dataFilePath, dataFile)
    
    try:
        with uproot.open(fullFilePath) as rootFile:
            dataframes = {}
            for treeName in rootFile.keys():
                if isinstance(rootFile[treeName], uproot.behaviors.TTree.TTree):
                    tree = rootFile[treeName]
                    try:
                        df = tree.arrays(library='pd')
                        dataframes[treeName] = df
                    except Exception as e:
                        print(f"Error reading tree '{treeName}': {e}")
            return dataframes
    except Exception as e:
        print(f"Error opening or reading ROOT file '{fullFilePath}': {e}")
        return {}

In [ ]:
def readMultipleRuns(runNumbers, treesToRead=None):
    """
    Reads the metatdata and desired simulation data from the specified runs.

    MetaData is always included in output. 
        If treesToRead is an empty list then only metadata is output.
        If treesToRead is None then all are output.
    The data is converted to pandas dataframes within a dictionary.
    The keys to this dict are the run numbers.

    Args:
        runNumbers (list): list of run numbers to be read.
        treesToRead (list): List of strings containing the desired tree name to read.

    Returns:
        allData (dict): Dictionary containing pandas dataframes with the simulation data.
    """
    #TODO - This function also exists in analysis.ipynb
    possibleDataTrees = [
        'fieldLineData',
        'gridFieldLineData',
        'electronData',
        'ionData',
        'avalancheData',
        'electronTrackData'
    ]

    filteredToRead = []
    if treesToRead is None:
        filteredToRead = possibleDataTrees
    elif not treesToRead:
        filteredToRead = []
    else:
        #Error check the tree parsing
        invalidTrees = []
        for inTree in treesToRead:
            if inTree == 'metaData':
                continue
            if inTree not in possibleDataTrees:
                invalidTrees.append(inTree)
            else:
                filteredToRead.append(inTree)
                
        if invalidTrees:
            raise ValueError(f"Cannot read the following trees: '{invalidTrees}'.")

    #Read in data for each run
    allData = {}
    for inRun in runNumbers:

        #Read the root tree for this run
        try:
            simData = readRootTrees(inRun)
        except FileNotFoundError:
            print(f"Warning: Could not find data for run {inRun} - Skipping.")
            continue

        allData[inRun] = {}

        #Always read the metadata, skip run if unsuccessful
        try:
            allData[inRun]['metaDataTree'] = simData['metaDataTree;1']
        except KeyError:
            print(f"Warning: 'metadata' not found in run {inRun}. Skipping this run.")
            allData[inRun]['metadata'] = None 
            continue

        #Read the remaining trees
        for inTree in filteredToRead:
            treeKey = f'{inTree}Tree;1'
            try:
                allData[inRun][inTree] = simData[treeKey]
            except KeyError:
                print(f"Warning: Tree '{inTree}' not found for run {inRun}.")
                allData[inRun][inTree] = None

    return allData

In [ ]:
def checkFieldTransparency(runNumber):
    """
    Checks the field transparency for a given simulation run number.
    Checks against the simulation's limit in its metadata.

    Args:
        runNumber (int): Run number.

    Returns:
        bool: True if tranparency is greater than the limit, False otherwise.
    
    """    
    simData = readMultipleRuns([runNumber], [])
    transparency = simData['metaData']['Field Transparency'].iloc[0]
    limit = simData['metaData']['Field Transparency Limit'].iloc[0]
    
    if transparency < limit:
        return False
    
    return True

In [ ]:
def transparencyScan(baseParam, scanParam, order=True, changeGeo=True):
    """
    Performs a 1-dimensional scan of the simulation,
    looking for the field transparency -limiting case.

    Sets all parameters based on baseParam, 
    then linearly scans through a specified parameter.
    If 'order' is True it scans in increasing order, descending if False.
    Simulations terminate if the transparency falls below the limit.

    Args:
        baseParam (dict): Dictionary containing
                          the base simulation parameters.
        scanParam (list): List of the form: 
                          ['parameterToScan', minValue, maxValue, numSteps]
                            - parameterToScan (str): The name of the parameter to vary.
                            - minValue (float): The minimum value for the scan.
                            - maxValue (float): The maximum value for the scan.
                            - numSteps (int): The number of steps in the scan.
        order (bool): Boolean that dictates the order of the scan. 
                      Ascending if True, descending if False.
        changeGeo (bool): Optional parameter to bypass geometry meshing.
                           Efficient if geometry does not change between runs.

    Returns:
        list: List of run numbers where the field transparency is greater than the limit.
              Empty if not all parameters are provided in baseParam or 
              if the scanning parameter is invalid
    """
    #Reset parameters before scan
    allParam = resetParam()

    for inParam in allParam:
        if inParam not in baseParam:
            print(f"Base parameter '{inParam}' not included.")
            return []

    if scanParam[0] not in allParam.keys():
        print('Invalid scanning parameter.')
        return []

    #Determine values for scan
    scanValues = np.linspace(scanParam[1], scanParam[2], scanParam[3])
    if not order:
        scanValues = np.flip(scanValues)
    
    runNos = []
    print(f'Beginning transparency scan for: {scanParam[0]}')
    print(f'\tTransparency limit is: {baseParam['transparencyLimit']}')
    for inValue in scanValues:
        baseParam[scanParam[0]] = inValue

        #Ensure geometry is generated on at least the first sim
        if inValue == scanValues[0]:
            runNo = runSimulation(baseParam)
        else:
            runNo = runSimulation(baseParam, changeGeo)

        if runNo == -1:
            print(f'An error occured at: {scanParam[0]} = {inValue}')
            break

        #Terminate scan if transparency is below limit
        if not checkFieldTransparency(runNo):
            print(f'Field transparency too low at: {scanParam[0]} = {inValue}. Ending scan')
            break

        runNos.append(runNo)

    #Done running - reset to defaults
    print(f'Done scan through {scanParam[0]}. Final run number: {getRunNumber()-1}')
    _ = resetParam()
    
    return runNos

In [ ]:
# Default run of a simulation.
## Determines a start-to-finish execution time.
## Individual program times from log files.
if __name__ == '__main__':
    defaultSim = FIMS_Simulation()
    startTime = time.monotonic()
    defaultSim.runSimulation()
    endTime = time.monotonic()
    print(f'Test run takes ~{endTime-startTime:.1f}s.')

    '''
    Last time Tanner ran this test:
    Total time: ~940 s
        Gmsh: ~62 s
        ElmerGrid: ~1 s   
        ElmerSolver: ~729 s
        ElmerWeighting: ~33 s
        Garfield: ~ 114 s
    '''
    '''
    Last time James ran this test:
    Total time: ~ s
        Gmsh: ~ s
        ElmerGrid: ~ s   
        ElmerSolver: ~ s
        ElmerWeighting: ~ s
        Garfield: ~ s
    '''

In [ ]:
#if __name__ == '__main__':
#    scanValues([
#        ['fieldRatio', 30, 60, 4],
#        ['holeRadius', 16, 24, 3],
#        ['gridStandoff', 90, 110, 3]
#    ])
'''
[5365,
 5366,
 5367,
 5368,
 5369,
 5370,
 5371,
 5372,
 5373,
 5374,
 5375,
 5376,
 5377,
 5378,
 5379,
 5380,
 5381,
 5382,
 5383,
 5384,
 5385,
 5386,
 5387,
 5388,
 5389,
 5390,
 5391,
 5392,
 5393,
 5394,
 5395,
 5396,
 5397,
 5398,
 5399,
 5400]
'''